# Conceptos de programación en TensorFlow

*El contenido de este cuaderno está basado en el material de ejemplo provisto en Google Colab y ha sido traducido y modificado parcialmente para los propósitos del curso de programación en Python del grupo de análisis del Banco de Guatemala. El enlace al cuaderno original, así como la licencia de uso de este cuaderno se muestran a continuación.* 
- [https://colab.research.google.com/notebooks/mlcc/tensorflow_programming_concepts.ipynb](https://colab.research.google.com/notebooks/mlcc/tensorflow_programming_concepts.ipynb)  

Copyright 2017 Google LLC.

In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

**Objetivos:**
  * Conocer los fundamentos del modelo de programación de TensorFlow, enfocándonos en los siguientes conceptos:
    * tensores
    * operaciones
    * grafos
    * sesiones
  * Construir un programa de TensorFlow que utilice estos elementos de forma ilustrativa, a modo de replicar el flujo común de utilización. 

## Resumen de conceptos

TensorFlow recibe su nombre de los **tensores**, que son arreglos de datos de dimensionalidad arbitraria. Con TensorFlow, puede manipular tensores con un número muy alto de dimensiones. Dicho esto, la mayoría de las veces trabajará con uno o más de los siguientes tensores de baja dimensión:

  * A ** escalar ** es una matriz 0-d (un tensor de orden 0). Por ejemplo, "Howdy" `o` 5`
  * Un ** vector ** es una matriz 1-d (un tensor de primer orden). Por ejemplo, `[2, 3, 5, 7, 11]` o `[5]`
  * Una ** matriz ** es una matriz de 2-d (un tensor de segundo orden). Por ejemplo, `[[3.1, 8.2, 5.9] [4.3, -2.7, 6.5]]`

Las operaciones **TensorFlow** crean, destruyen y manipulan tensores. La mayoría de las líneas de código en un programa TensorFlow típico son operaciones.

In [1]:
# %tensorflow_version 1.x
# import tensorflow.compat.v1 as tf
import tensorflow as tf
tf.__version__

'1.15.0'

### Gráfico computacional

Un gráfico ** de TensorFlow ** (también conocido como ** gráfico computacional ** o ** gráfico de flujo de datos **) es, sí, una estructura de datos de gráfico. Los nodos de un gráfico son operaciones (en TensorFlow, cada operación está asociada a un gráfico). Muchos programas TensorFlow consisten en un solo gráfico, pero los programas TensorFlow pueden crear opcionalmente múltiples gráficos. 
- Los nodos de un gráfico son operaciones; 
- Los bordes de un gráfico son tensores. Los tensores fluyen a través del gráfico, manipulados en cada nodo por una operación. 
- El tensor de salida de una operación a menudo se convierte en el tensor de entrada de una operación posterior. 
- TensorFlow implementa un ** modelo de ejecución diferida**, lo que significa que los nodos solo se calculan cuando es necesario, en función de las necesidades de los nodos asociados.

Los tensores se pueden almacenar en el gráfico como ** constantes ** o **variables**. Como es de suponer, las constantes contienen tensores cuyos valores no pueden cambiar, mientras que las variables contienen tensores cuyos valores pueden cambiar. Sin embargo, lo que quizás no resulta tan intuitivo es que las constantes y las variables ¡son solo más operaciones en el gráfico!.  

- Una constante es una operación que siempre devuelve el mismo valor tensorial. 
- Una variable es una operación que devolverá el tensor que se le haya asignado.


### Constantes

Para definir una constante, use el operador `tf.constant` y pase su valor. Por ejemplo:

In [2]:
x = tf.constant(5.2)
x

<tf.Tensor 'Const:0' shape=() dtype=float32>

### Variables

Del mismo modo, puede crear una variable como esta:

In [3]:
y = tf.Variable([5])
y

<tf.Variable 'Variable:0' shape=(1,) dtype=int32_ref>

También es posible crear la variable primero y luego asignar un valor como este (tomar en cuenta que siempre debe especificar un valor predeterminado):

In [4]:
y = tf.Variable([0]) 
y = y.assign([5]) 
y

<tf.Tensor 'Assign:0' shape=(1,) dtype=int32_ref>

In [5]:
z = tf.Variable(tf.random.truncated_normal(shape=(5,5)))
z

<tf.Variable 'Variable_2:0' shape=(5, 5) dtype=float32_ref>

Una vez que haya definido algunas constantes o variables, puede combinarlas con otras operaciones como `tf.add`. Cuando evalúa la operación tf.add, llamará a sus operaciones `tf.constant` o `tf.Variable` para obtener sus valores y luego devolver un nuevo tensor con la suma de esos valores.

Los gráficos deben ejecutarse dentro de una sesión **TensorFlow**, que contiene el estado de los gráficos que ejecuta:

In [6]:
with tf.Session() as sess:
  initialization = tf.global_variables_initializer()
  print(y.eval())

[5]


Al trabajar con `tf.Variable`s, se deben inicializar explícitamente llamando creando un nodo inicializador en el gráfico con la función ` tf.global_variables_initializer` al comienzo de la sesión, como se muestra arriba.

** Nota: ** Una sesión puede distribuir la ejecución de gráficos en varias máquinas (suponiendo que el programa se ejecute en un marco de cómputo distribuido). Para obtener más información, consulte [TensorFlow distribuido] (https://www.tensorflow.org/deploy/distributed).

### Resumen

La programación de TensorFlow es esencialmente un proceso de dos pasos:

  1. Crear constantes, variables y definir las operaciones en un gráfico.
  2. Evaluar esas constantes, variables y operaciones dentro de una sesión.

## Creando un programa muy sencillo de TensorFlow

Veamos cómo podemos hacer un programa que se encargue de sumar dos constantes.

In [8]:
import tensorflow as tf
tf.__version__

'1.15.0'

**No olvidar ejecutar el bloque de código anterior (con `import` las sentencias de `import`).**

Otros módulos que es común importar al utilizar TensorFlow son:

```
  import matplotlib.pyplot as plt # Visualización de datos.
  import numpy as np              # Librería numérica de bajo nivel.
  import pandas as pd             # Librería numérica de alto nivel.
```

TensorFlow proporciona un **gráfico predeterminado** por defecto. Sin embargo, es recomendable crear explícitamente un `tf.Graph` propio para facilitar el manejo de los cómputos (por ejemplo, es posible que desee trabajar con un `Graph` diferente en cada celda de este cuaderno).

In [12]:
# %tensorflow_version 1.x
import tensorflow as tf

# Crear un gráfico
g = tf.Graph()

# Establecer g como el gráfico por defecto
with g.as_default():
  # Armar el gráfico con las siguientes tres operaciones:
  #   * Dos definiciones de tf.constant para los operandos.
  #   * Una operación tf.add para sumar los operandos.
  x = tf.constant(8, name="x_const")
  y = tf.constant(5, name="y_const")
  my_sum = tf.add(x, y, name="xy_sum")


  # Ahora creamos una sesión con tf.Session
  # Por defecto, la sesión corre el gráfico por defecto (el que acabamos de definir)`graph`:
  with tf.Session() as sess:
    print(my_sum.eval())

13


In [14]:
# También es posible indicar el gráfico con el argumento 
with tf.Session(graph = g) as sess:
    print(my_sum.eval())

13


## Ejercicio: Introducir un tercer operando

Modificar el código anterior para sumar tres constantes, en lugar de dos: 

  1. Definir una tercera constante escalar, `z`, y asignarle el valor de `4`.
  2. Sumar `z` con el tensor `my_sum` para tener el nuevo resultado.
  
  **Tip:** Vea la documentación del API [tf.add()](https://www.tensorflow.org/api_docs/python/tf/add) para más detalles sobre la función `tf.add`.
  
  3. Reejecutar el bloque de código. ¿Se genera el total de los tres números?

### Solución

Ver la solución abajo

In [15]:
# Create a graph.
g = tf.Graph()

# Establish our graph as the "default" graph.
with g.as_default():
  # Assemble a graph consisting of three operations. 
  # (Creating a tensor is an operation.)
  x = tf.constant(8, name="x_const")
  y = tf.constant(5, name="y_const")
  my_sum = tf.add(x, y, name="x_y_sum")
  
  # Task 1: Define a third scalar integer constant z.
  z = tf.constant(4, name="z_const")
  # Task 2: Add z to `my_sum` to yield a new sum.
  new_sum = tf.add(my_sum, z, name="x_y_z_sum")

  # Now create a session.
  # The session will run the default graph.
  with tf.Session() as sess:
    # Task 3: Ensure the program yields the correct grand total.
    print(new_sum.eval())

17


## Uso de los *placeholders*

En muchos ejemplos es común la utilización de *placeholders*, que permiten definir tensores para los cuales no proveemos ningún valor por defecto hasta que ejecutemos el gráfico dentro de una sesión. Un ejemplo sería el siguiente:

In [37]:
tf.reset_default_graph()

# Creamos los parámetros de una función lineal
m = tf.Variable(tf.random.normal(shape=[100, 1]))
b = tf.Variable(0.5)

# Definimos un placeholder para los "datos" que vamos a alimentar
x = tf.placeholder(tf.float32, shape=[None, 100])

# Definimos el modelo
y = x@m + b
y

<tf.Tensor 'add:0' shape=(?, 1) dtype=float32>

Ahora vamos a crear nuestros datos, que en este ejemplo será un vector fijo lleno de unos y evaluaremos nuestra función lineal en estos valores dentro de una sesión de TensorFlow. En la mayoría de aplicaciones, estos datos los tenemos, los cargamos utilizando pandas u otra librería. 

In [34]:
import numpy as np
X_data = np.random.rand(100, 100)
# X_data

In [38]:
init = tf.global_variables_initializer()

# Ahora obtenemos los valores de nuestra función lineal
with tf.Session() as sess:
    sess.run(init)
    valores_y = sess.run(y, feed_dict={x: X_data})

# Mostrar valores computados
valores_y

array([[-1.4323144 ],
       [ 1.3766531 ],
       [ 0.8874216 ],
       [ 4.8408794 ],
       [ 6.057294  ],
       [ 1.0431738 ],
       [-1.7387359 ],
       [ 6.704713  ],
       [ 2.2794044 ],
       [ 3.3965192 ],
       [ 0.19536322],
       [ 0.7329569 ],
       [ 0.2920779 ],
       [-0.33746612],
       [ 2.1109288 ],
       [ 1.5155232 ],
       [ 3.1489534 ],
       [ 4.830813  ],
       [ 4.833971  ],
       [-3.607377  ],
       [-1.396883  ],
       [ 1.2371359 ],
       [ 3.656588  ],
       [-0.59370065],
       [-2.7882712 ],
       [ 0.12208796],
       [ 0.9389446 ],
       [ 1.5555031 ],
       [ 4.231166  ],
       [ 3.184585  ],
       [ 5.876491  ],
       [-0.26570535],
       [ 3.1747146 ],
       [ 1.2581273 ],
       [-2.7460213 ],
       [-3.3312516 ],
       [ 2.7264965 ],
       [ 3.7423296 ],
       [ 4.2903905 ],
       [ 3.0173845 ],
       [ 1.0961123 ],
       [-0.60179293],
       [ 1.8987813 ],
       [-0.42837906],
       [ 2.8517385 ],
       [ 4